In [1]:
from genetic_algorithm import *
import re
import json

## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 4

res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## Sumuj duble

In [ ]:
dic = sum_duplicated_neurons(res)

## brain generator

In [4]:
edges = dic[1]
def generate_brain_output_dictionary(edges):
    '''generate list of outputs dictionary to store 'mid' and 'in' neurons'''
    brain = {}
    for i in edges:
        if 'out' in edges[i][1]:
            brain.update({edges[i][1]:{}})

In [5]:
mid_neuron(brain, edges)
print(brain)

analyze_brain(brain)

{'0out': {'2in': {'w': -3.2361361208606745},
  '1mid': {'w': -0.9595605066381809,
   '0mid': {'w': -0.26174271326110177, '0in': {'w': -3.582725469250725}},
   '1mid': {'w': -0.072272241721349},
   '1in': {'w': -1.6096444376621395}}},
 '1out': {'0in': {'w': 1.107034945826339}},
 '2out': {'1mid': {'w': 1.738806653441172,
   '0mid': {'w': -0.26174271326110177, '0in': {'w': -3.582725469250725}},
   '1mid': {'w': -0.072272241721349},
   '1in': {'w': -1.6096444376621395}},
  '0mid': {'w': -0.7606897604150771, '0in': {'w': -3.582725469250725}}}}

In [6]:

            
analyze_brain(brain)

if 0out {'2in': {'w': -3.2361361208606745}, '1mid': {'w': -0.9595605066381809, '0mid': {'w': -0.26174271326110177, '0in': {'w': -3.582725469250725}}, '1mid': {'w': -0.072272241721349}, '1in': {'w': -1.6096444376621395}}}
else 1out {'0in': {'w': 1.107034945826339}}
if 2out {'1mid': {'w': 1.738806653441172, '0mid': {'w': -0.26174271326110177, '0in': {'w': -3.582725469250725}}, '1mid': {'w': -0.072272241721349}, '1in': {'w': -1.6096444376621395}}, '0mid': {'w': -0.7606897604150771, '0in': {'w': -3.582725469250725}}}


get dictionary depth  
https://stackoverflow.com/questions/23499017/know-the-depth-of-a-dictionary
https://www.tutorialspoint.com/find-depth-of-a-dictionary-in-python

https://stackoverflow.com/questions/47265412/custom-connections-between-layers-keras